In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive


In [ ]:
import os

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.16 MiB | 20.25 MiB/s, done.
Resolving deltas: 100% (10405/10405), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [ ]:
# update the frequency at which model weights are saved, by directly modifying darknet source code. 
# (could be done manually by selecting and modifying the file on the left window of google colab)
with open('src/detector.c', "r+") as f:
  old = f.read()
  selec = '(iteration >= (iter_save + 1000) || iteration % 1000 == 0) && net.max_batches < 10000)'
  s1, s2 = old.split(selec)
  new = s1 + '(iteration >= (iter_save + 500) || iteration % 500 == 0) && net.max_batches < 1000000)' + s2
  f.seek(0) # rewind
  f.write(new)

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
# copy over both datasets into the root directory of the Colab VM
!cp /mydrive/deepberry/train.zip ../
!cp /mydrive/deepberry/valid.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../train.zip -d data/
!unzip ../valid.zip -d data/

In [ ]:
# .CFG 

# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/deepberry/detection.cfg ./cfg

In [ ]:
# create classes.names file
with open('/mydrive/deepberry/classes.names', 'w') as out:
  out.write('berry') # the class name doesn't matter since there is only one class

# load it
!cp /mydrive/deepberry/classes.names ./data

In [ ]:
# create backup folder (where model weights are saved during training)
if not os.path.isdir('/mydrive/deepberry/backup'):
  os.mkdir('/mydrive/deepberry/backup')

In [ ]:
# create training.data file
with open('/mydrive/deepberry/training.data', 'w') as out:
  out.write('classes = 1\ntrain = data/train.txt\nvalid = data/valid.txt\nnames = data/classes.names\nbackup = /mydrive/deepberry/backup')

# load it
!cp /mydrive/deepberry/training.data  ./data

In [ ]:
# create train.txt and valid.txt files

with open('/mydrive/deepberry/train.txt', 'w') as out:
  images = [f for f in os.listdir('data/train') if f[-4:] == '.png']
  for f in images:
    out.write('data/train/{}\n'.format(f))

with open('/mydrive/deepberry/valid.txt', 'w') as out:
  images = [f for f in os.listdir('data/valid') if f[-4:] == '.png']
  for f in images:
    out.write('data/valid/{}\n'.format(f))

# load it
!cp /mydrive/deepberry/train.txt  ./data
!cp /mydrive/deepberry/valid.txt  ./data

In [ ]:
# # Train the model (transfer learning with pre-trained weights)
# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
# !./darknet detector train data/training.data cfg/yolov4-config-tiny.cfg yolov4-tiny.conv.29 -dont_show -map | tee results.txt

In [ ]:
# Train the model from scratch. 
# Weights will be saved in the backup folder.

!./darknet detector train data/training.data cfg/detection.cfg -dont_show -map | tee results.txt

In [ ]:
# Continue training starting from the last weights saved
!./darknet detector train data/training.data cfg/detection.cfg /mydrive/deepberry/backup/detection_last.weights -dont_show -map | tee results.txt